# Laserfarm: LiDAR point cloud analysis for macro-ecology

## Configuration

### User parameters

Defines the parameters that can be set by users when executing the workflow.

In [ ]:
# (DO NOT containerize this cell)

# Data handling parameters
param_minio_server = ''
param_bucket_name = ''
param_remote_path_root = ''

# Laserfarm parameters
param_feature_name = 'perc_95_normalized_height'
param_validate_precision = '0.001'
param_tile_mesh_size = '10.'
param_filter_type = 'select_equal'
param_attribute = 'raw_classification'
param_min_x = '-113107.81'  # EPSG:28992
param_max_x = '398892.19'  # EPSG:28992
param_min_y = '214783.87'  # EPSG:28992
param_max_y = '726783.87'  # EPSG:28992
param_n_tiles_side = '512'
param_apply_filter_value = '1'
param_laz_compression_factor = '7'
param_max_filesize_mb = '80'  # desired max file size (in MiB)


### Dependencies

The following cells install extra dependencies that are not included in the Laserfarm flavor by default, and import the libraries used in the notebook.

In [ ]:
# (DO NOT containerize this cell)

!pip install minio rasterio

In [ ]:
# (DO NOT containerize this cell)

import json
import os

from laserfarm import DataProcessing, GeotiffWriter, Retiler
from laserfarm.remote_utils import get_wdclient, list_remote
from minio import Minio
import laspy

### Global configuration

The following variable are used throughout the code. They are intended to be edited by developers who the notebook.

In [ ]:
# (DO NOT containerize this cell)

conf_local_tmp = '/tmp/data'
conf_local_path_raw = os.path.join(conf_local_tmp, 'raw')
conf_local_path_split = os.path.join(conf_local_tmp, 'split')
conf_local_path_retiled = os.path.join(conf_local_tmp, 'retiled')
conf_local_path_targets = os.path.join(conf_local_tmp, 'targets')
conf_local_path_geotiff = os.path.join(conf_local_tmp, 'geotiff')
conf_local_path_figures = os.path.join(conf_local_tmp, 'figures')

## Workflow steps

### Fetch laz files from remote storage

This cell downloads `.laz` files from the remote MinIO storage.

In [ ]:
# S1 Fetch laz files

os.makedirs(conf_local_path_raw, exist_ok=True)
raw_laz_files = []

minio_client = Minio(param_minio_server, secure=True)
objects = minio_client.list_objects(
    param_bucket_name, prefix=param_remote_path_root
    )
for obj in objects:
    if obj.object_name.lower().endswith('.laz'):
        laz_file = os.path.join(
            conf_local_path_raw, obj.object_name.split('/')[-1]
            )
        if not os.path.isfile(laz_file):
            print(
                f'Downloading {param_bucket_name}:{obj.object_name} to {laz_file}'
                )
            minio_client.fget_object(
                param_bucket_name, obj.object_name, laz_file
                )
        else:
            print(
                f'Skipping download of {param_bucket_name}:{obj.object_name} because {laz_file} already exists'
                )
        raw_laz_files.append(laz_file)

print(raw_laz_files)

### Split big files

This step splits big laz files into smaller files. This is useful when the original files are too large for the VMs used for the processing. If the VMs have enough memory to hold the laz files, this step can be skipped.

In [ ]:
# S2 Split big laz

import os


def save_chunk_to_laz_file(
        in_filename,
        out_filename,
        offset,
        n_points
        ):
    """ Read points from a LAS/LAZ file and write them to a new file. """
    with laspy.open(in_filename) as in_file:
        with laspy.open(
                out_filename,
                mode="w",
                header=in_file.header
                ) as out_file:
            in_file.seek(offset)
            points = in_file.read_points(n_points)
            out_file.write_points(points)
    return out_filename


def split_strategy(filename, max_filesize, dest_dir=None):
    """ Set up splitting strategy for a LAS/LAZ file. """
    with laspy.open(filename) as f:
        bytes_per_point = (
                f.header.point_format.num_standard_bytes +
                f.header.point_format.num_extra_bytes
        )
        n_points = f.header.point_count
    n_points_target = int(
        max_filesize * int(param_laz_compression_factor) / bytes_per_point
        )
    stem, ext = os.path.splitext(filename)
    if dest_dir is not None:
        stem = os.path.join(dest_dir, os.path.basename(stem))
    return [
        (filename, f"{stem}-{n}{ext}", offset, n_points_target)
        for n, offset in enumerate(range(0, n_points, n_points_target))
        ]


os.makedirs(conf_local_path_split, exist_ok=True)
split_laz_files = []

for raw_file in raw_laz_files:
    inps = split_strategy(
        raw_file,
        int(param_max_filesize_mb) * 2 ** 20,
        dest_dir=conf_local_path_split,
        )
    print(f'Splitting {raw_file} into {len(inps)} files:')
    for inp in inps:
        split_file = inp[1]
        if not os.path.isfile(split_file):
            print(f'  writing {split_file}')
            save_chunk_to_laz_file(*inp)
        else:
            print(f' skipping {split_file} because it already exists')
        split_laz_files.append(split_file)

print(split_laz_files)

### Retile laz files

This step splits the laz files into small tiles that can be easily processed.

In [ ]:
# S3 Retile laz files

grid_retile = {
    'min_x': float(param_min_x),
    'max_x': float(param_max_x),
    'min_y': float(param_min_y),
    'max_y': float(param_max_y),
    'n_tiles_side': int(param_n_tiles_side),
    }

retiling_input = {
    'setup_local_fs': {
        'input_folder': conf_local_path_split,
        'output_folder': conf_local_path_retiled,
        },
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    }

os.makedirs(conf_local_path_retiled, exist_ok=True)
tiles = []

for file in split_laz_files:
    base_name = os.path.splitext(os.path.basename(file))[0]
    retile_record_filename = os.path.join(
        conf_local_path_retiled,
        f'{base_name}_retile_record.js',
        )
    if not os.path.isfile(retile_record_filename):
        print(f'Retiling {file}')
        retiler = Retiler(file, label=file).config(retiling_input)
        retiler.run()
    else:
        print(
            f'Skipping retiling of {file} because {retile_record_filename} already exists'
            )
    # load filenames from retile record
    with open(retile_record_filename, 'r') as f:
        retile_record = json.load(f)
    tiles += retile_record['redistributed_to']

print(tiles)

### Unique tiles

Ensure that the list of tiles contains no duplicates. This is necessary because the above step might return duplicates. If no splitting is used, this cell can be merged with the above one. 

In [ ]:
# S4 Unique tiles

tiles = list(set(tiles))

print(tiles)

### Extract features from tiles

Run the feature extraction for each tile. The features are extracted using [laserchicken](https://github.com/eEcoLiDAR/laserchicken).

In [ ]:
# S5 Extract features

feature_files = []

for i, tile in enumerate(tiles):
    grid_feature = {
        'min_x': float(param_min_x),
        'max_x': float(param_max_x),
        'min_y': float(param_min_y),
        'max_y': float(param_max_y),
        'n_tiles_side': int(param_n_tiles_side),
        }

    feature_extraction_input = {
        'setup_local_fs': {
            'input_folder': conf_local_path_retiled,
            'output_folder': conf_local_path_targets,
            },
        'load': {'attributes': [param_attribute]},
        'normalize': 1,
        'apply_filter': {
            'filter_type': param_filter_type,
            'attribute': param_attribute,
            'value': [int(param_apply_filter_value)],
            #ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
            },
        'generate_targets': {
            'tile_mesh_size': float(param_tile_mesh_size),
            'validate': True,
            'validate_precision': float(param_validate_precision),
            **grid_feature
            },
        'extract_features': {
            'feature_names': [param_feature_name],
            'volume_type': 'cell',
            'volume_size': float(param_tile_mesh_size),
            },
        'export_targets': {
            'attributes': [param_feature_name],
            'multi_band_files': False,
            },
        }
    idx = (tile.split('_')[1:])

    target_file = os.path.join(
        conf_local_path_targets, param_feature_name, tile + '.ply'
        )
    print(target_file)

    if not os.path.isfile(target_file):
        print(f'Extracting features from {tile} ({i + 1} of {len(tiles)})')
        processing = DataProcessing(tile, tile_index=idx, label=tile).config(
            feature_extraction_input
            )
        processing.run()
    else:
        print(
            f'Skipping features extraction for {tile} ({i + 1} of {len(tiles)}) because {target_file} already exists'
            )

    feature_files.append(target_file)

print(feature_files)

### Save GeoTIFF

Generate a GeoTIFF containing the extracted features. This merges all tiles into one file. 

In [ ]:
# S6 Save GeoTIFF

print(feature_files)

geotiff_export_input = {
    'setup_local_fs': {
        'input_folder': conf_local_path_targets,
        'output_folder': conf_local_path_geotiff,
        },
    'parse_point_cloud': {},
    'data_split': {
        'xSub': 1,
        'ySub': 1,
        },
    'create_subregion_geotiffs': {
        'output_handle': 'geotiff'
        },
    }

writer = (
    GeotiffWriter(
        input_dir=param_feature_name,
        bands=param_feature_name,
        label=param_feature_name,
        )
    .config(geotiff_export_input)
    )
writer.run()

geo_tiff = os.path.join(
    conf_local_path_geotiff,
    f'geotiff_TILE_000_BAND_{param_feature_name}.tif'
    )
print(geo_tiff)

### Create figures

In [ ]:
# S7 Figures

import matplotlib.pyplot as plt
import rasterio
import rasterio.plot as rp

src = rasterio.open(geo_tiff)

os.makedirs(conf_local_path_figures, exist_ok=True)

ax = plt.gca()
rio_plot = rp.show((src, 1), interpolation='none', ax=ax)
img = rio_plot.get_images()[0]
cb = plt.colorbar(img, ax=ax)
cb.set_label(f'{param_feature_name}')
plt.xlabel('EPSG:28992 X [m]')
plt.ylabel('EPSG:28992 Y [m]')
plt.show()
plt.savefig(os.path.join(conf_local_path_figures, f'{param_feature_name}_map.pdf'))

rp.show_hist(
    src,
    bins=50,
    lw=0.0,
    stacked=False,
    alpha=0.3,
    histtype='stepfilled',
    title="Histogram",
    )
plt.show()
plt.savefig(os.path.join(conf_local_path_figures, f'{param_feature_name}_histogram.pdf'))